In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!ls ../raw_data/

holidays_events.csv    test.csv		     train2017_perish.csv
items.csv	       train2013_perish.csv  train_all_table.csv
oil.csv		       train2014_perish.csv  train.csv
sample_submission.csv  train2015_perish.csv  train_items_stores.csv
stores.csv	       train2016.csv	     train_stores_holiday.csv
stores_sales	       train2016_perish.csv  transactions.csv


In [3]:
import numpy as np
import pandas as pd

# Read and init data

In [4]:
# columns Unnamed: 0 drop
train_data = pd.read_csv('../raw_data/train2016.csv')

In [6]:
train_data.drop(columns='Unnamed: 0', inplace=True)

---

In [9]:
train_itemsp = pd.read_csv('../raw_data/train2016_perish.csv')

In [11]:
train_itemsp.drop(columns='Unnamed: 0', inplace=True)

In [73]:
test_data = pd.read_csv('../raw_data/test.csv')

In [4]:
holiday_data = pd.read_csv('../raw_data/holidays_events.csv')
oil_data = pd.read_csv('../raw_data/oil.csv')

stores_data = pd.read_csv('../raw_data/stores.csv')
transactions_data = pd.read_csv('../raw_data/transactions.csv')

items_data = pd.read_csv('../raw_data/items.csv')

In [ ]:
# if we want to use
train_stores_holiday = pd.read_csv('../raw_data/train_stores_holiday.csv')
train_items_stores = pd.read_csv('../raw_data/train_items_stores.csv')

train_all_table = pd.read_csv('../raw_data/train_all_table.csv')

In [74]:
test_data.shape # (3370464, 5)

(3370464, 5)

---

# Load train data at 1 year

Date is an object (Convert datetime)

In [74]:
train_data['date'] = pd.to_datetime(train_data['date'])

In [76]:
train_data['year'] = train_data['date'].dt.year

In [78]:
train_data['month'] = train_data['date'].dt.month

In [79]:
train_data['day'] = train_data['date'].dt.day

In [80]:
train_data['dayofweek'] = train_data['date'].dt.dayofweek

In [81]:
train_data_2017 = train_data[train_data['year'] == 2017]
train_data_2017

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year,month,day,dayofweek


---

# Convert all date to datetime

In [112]:
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# oil_data['date'] = pd.to_datetime(oil_data['date'])

transactions_data['date'] = pd.to_datetime(transactions_data['date'])

Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object')

In [60]:
holiday_data['date'] = pd.to_datetime(holiday_data['date'])

In [61]:
train_itemsp['date'] = pd.to_datetime(train_itemsp['date'])

# Train data

---

# Oil data

--- 

# Holiday data

## Merge Preparation

In [ ]:
# type, locale, locale_name

In [6]:
holiday_data.columns

Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')

In [25]:
# transfered : if day of week which are holiday (like SUNDAY) is moved
holiday_data['transferred'].value_counts()

False    338
True      12
Name: transferred, dtype: int64

In [28]:
holiday_data['type'].value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: type, dtype: int64

In [27]:
holiday_data['locale'].value_counts()

National    174
Local       152
Regional     24
Name: locale, dtype: int64

### Check `locale_name` with `city` and `state`

In [8]:
holiday_data['locale_name'].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato',
       'Ibarra', 'Quevedo', 'Santo Domingo de los Tsachilas',
       'Santa Elena', 'Quito', 'Loja', 'Salinas', 'Guayaquil'],
      dtype=object)

In [17]:
len(holiday_data['locale_name'].unique())

24

In [11]:
stores_data['city'].unique()

array(['Quito', 'Santo Domingo', 'Cayambe', 'Latacunga', 'Riobamba',
       'Ibarra', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil', 'Salinas',
       'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad', 'Cuenca',
       'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'],
      dtype=object)

In [24]:
print('count city', len(stores_data['city'].unique()))

count city 22


In [15]:
stores_data['state'].unique()

array(['Pichincha', 'Santo Domingo de los Tsachilas', 'Cotopaxi',
       'Chimborazo', 'Imbabura', 'Bolivar', 'Pastaza', 'Tungurahua',
       'Guayas', 'Santa Elena', 'Los Rios', 'Azuay', 'Loja', 'El Oro',
       'Esmeraldas', 'Manabi'], dtype=object)

In [23]:
print('Count state', len(stores_data['state'].unique()))

Count state 16


### Check if `city`, `state` and others in `holiday['locale_name']`

In [6]:
diff_city = stores_data['city'].unique()
diff_state = stores_data['state'].unique()

list_city_here = []
list_state_here = []

missing = []
for one_local_name in holiday_data['locale_name'].unique():
    if one_local_name in diff_city:
        list_city_here.append(one_local_name)
    elif one_local_name in diff_state:
        list_state_here.append(one_local_name)
    else:
        missing.append(one_local_name)
    
print('list city', list_city_here, '\n')
print('list state', list_state_here, '\n')
print('list missing', missing)

list city ['Manta', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo', 'Guaranda', 'Latacunga', 'Machala', 'Santo Domingo', 'El Carmen', 'Cayambe', 'Esmeraldas', 'Ambato', 'Ibarra', 'Quevedo', 'Quito', 'Loja', 'Salinas', 'Guayaquil'] 

list state ['Cotopaxi', 'Imbabura', 'Santo Domingo de los Tsachilas', 'Santa Elena'] 

list missing ['Ecuador']


In [10]:
check_city = []
check_state = []
check_ecuador_city = []
check_ecuador_state = []

for city in stores_data['city']:
    if city not in diff_city:
        check_city.append(city)
    if city == 'Ecuador':
        check_ecuador_city.append()

for state in stores_data['state']:
    if state not in diff_state:
        check_state.append(state)
    if state == 'Ecuador':
        check_ecuador_state.append()
        
        
print(check_city, '\n')
print(check_state, '\n')

print(check_ecuador_city, '\n')
print(check_ecuador_state)

[] 

[] 

[] 

[]


**Pas de city ou state == Ecuador** ...

### Check if Local = city ; Regional = state and National == 'Ecuador'

In [37]:
diff_city = stores_data['city'].unique()
diff_state = stores_data['state'].unique()

df_check = pd.DataFrame()
for index, holiday in holiday_data.iterrows():
#     print(holiday['locale_name'])
    if holiday['locale'] == 'Local' and holiday['locale_name'] in diff_city:
        pass
    elif holiday['locale'] == 'Regional' and holiday['locale_name'] in diff_state:
        pass
    elif holiday['locale'] == 'National' and holiday['locale_name'] == 'Ecuador':
        pass
    else:
        df_check = df_check.append(holiday, ignore_index = True)
                                         
print(df_check)
print(df_check.shape)

Empty DataFrame
Columns: []
Index: []
(0, 0)


**Local and Regional (And National) matches !**

### **Merge train stores with holiday**

In [76]:
train_data.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year
0,66458908,2016-01-01,25,105574,12.0,False,2016
1,66458909,2016-01-01,25,105575,9.0,False,2016
2,66458910,2016-01-01,25,105857,3.0,False,2016
3,66458911,2016-01-01,25,108634,3.0,False,2016
4,66458912,2016-01-01,25,108701,2.0,True,2016


In [77]:
train_data.shape # (35229871, 7)

(35229871, 7)

In [8]:
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [11]:
# Merge train with stores
train_stores = train_data.merge(stores_data, on='store_nbr')

train_stores

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year,city,state,type,cluster
0,66458908,2016-01-01,25,105574,12.0,False,2016,Salinas,Santa Elena,D,1
1,66458909,2016-01-01,25,105575,9.0,False,2016,Salinas,Santa Elena,D,1
2,66458910,2016-01-01,25,105857,3.0,False,2016,Salinas,Santa Elena,D,1
3,66458911,2016-01-01,25,108634,3.0,False,2016,Salinas,Santa Elena,D,1
4,66458912,2016-01-01,25,108701,2.0,True,2016,Salinas,Santa Elena,D,1
...,...,...,...,...,...,...,...,...,...,...,...
35229866,101688774,2016-12-31,54,2048381,6.0,True,2016,El Carmen,Manabi,C,3
35229867,101688775,2016-12-31,54,2048421,4.0,False,2016,El Carmen,Manabi,C,3
35229868,101688776,2016-12-31,54,2048471,10.0,False,2016,El Carmen,Manabi,C,3
35229869,101688777,2016-12-31,54,2048515,3.0,False,2016,El Carmen,Manabi,C,3


In [80]:
train_stores.shape # (35229871, 11)

(35229871, 11)

#### Manage `transferred`

In [16]:
# display to check 'type'
holiday_data.loc[holiday_data['transferred'] == True]

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


* Search `transferred == True`
* Search rows which matches with `transferred` (`type == 'Transfer'`)
* Remove rows which `transferred == True`

In [17]:
holiday_data.drop(holiday_data.loc[holiday_data['transferred'] == True].index, inplace=True)

In [18]:
holiday_data.loc[holiday_data['transferred'] == True] # not found normally

,date,type,locale,locale_name,description,transferred


In [19]:
holiday_data.loc[holiday_data['type'] == 'Transfer'] # keep this

,date,type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
256,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
269,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
304,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,False
313,2017-05-26,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
325,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


#### Create table `city_state` with `stores`

In [11]:
holiday_data.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [12]:
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [20]:
city_state = stores_data[['city', 'state']].drop_duplicates()
city_state.head()

,city,state
0,Quito,Pichincha
4,Santo Domingo,Santo Domingo de los Tsachilas
10,Cayambe,Pichincha
11,Latacunga,Cotopaxi
13,Riobamba,Chimborazo


**3 values : Local, National Regional**

#### Prepare local only

In [21]:
local_holiday = holiday_data.loc[holiday_data.locale == 'Local']

# new column city in local_holiday, useful for merging
local_holiday['city'] = local_holiday['locale_name']
local_holiday.head()

/tmp/ipykernel_13511/500300261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_holiday['city'] = local_holiday['locale_name']


,date,type,locale,locale_name,description,transferred,city
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,Manta
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,Cuenca
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,Libertad
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,Riobamba
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,Puyo


#### Prepare regional only

In [22]:
regional_holiday = holiday_data.loc[holiday_data.locale == 'Regional']

In [23]:
regional_holiday = regional_holiday.merge(city_state, left_on='locale_name', right_on='state')
regional_holiday.head()

,date,type,locale,locale_name,description,transferred,city,state
0,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga,Cotopaxi
1,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga,Cotopaxi
2,2014-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga,Cotopaxi
3,2015-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga,Cotopaxi
4,2016-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga,Cotopaxi


In [24]:
# state not useful for merging
regional_holiday.drop(columns='state', inplace=True)

In [25]:
regional_holiday.head()

,date,type,locale,locale_name,description,transferred,city
0,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga
1,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga
2,2014-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga
3,2015-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga
4,2016-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Latacunga


#### Prepare National only

In [26]:
national_holiday = holiday_data.loc[holiday_data.locale == 'National']
national_holiday.head()

,date,type,locale,locale_name,description,transferred
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False


In [27]:
city_state.head()

,city,state
0,Quito,Pichincha
4,Santo Domingo,Santo Domingo de los Tsachilas
10,Cayambe,Pichincha
11,Latacunga,Cotopaxi
13,Riobamba,Chimborazo


In [28]:
# Add column country in city_state to merge easily after
city_state['country'] = 'Ecuador'
city_state.head()

,city,state,country
0,Quito,Pichincha,Ecuador
4,Santo Domingo,Santo Domingo de los Tsachilas,Ecuador
10,Cayambe,Pichincha,Ecuador
11,Latacunga,Cotopaxi,Ecuador
13,Riobamba,Chimborazo,Ecuador


In [29]:
national_holiday = national_holiday.merge(city_state, left_on='locale_name', right_on='country')
national_holiday.head()

,date,type,locale,locale_name,description,transferred,city,state,country
0,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,Quito,Pichincha,Ecuador
1,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,Santo Domingo,Santo Domingo de los Tsachilas,Ecuador
2,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,Cayambe,Pichincha,Ecuador
3,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,Latacunga,Cotopaxi,Ecuador
4,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,Riobamba,Chimborazo,Ecuador


In [30]:
national_holiday.drop(columns=['state', 'country'], inplace=True)

#### `pd.concat` each `locale` with 3 previous variables

In [31]:
# local_holiday
# regional_holiday
# national_holiday

all_holiday = pd.concat([local_holiday, regional_holiday, national_holiday])[['date', 'city']].drop_duplicates()
all_holiday['is_holiday'] = 1
all_holiday

,date,city,is_holiday
0,2012-03-02,Manta,1
2,2012-04-12,Cuenca,1
3,2012-04-14,Libertad,1
4,2012-04-21,Riobamba,1
5,2012-05-12,Puyo,1
...,...,...,...
3647,2017-12-26,Loja,1
3648,2017-12-26,Machala,1
3649,2017-12-26,Esmeraldas,1
3650,2017-12-26,Manta,1


#### Merge `train_stores` with `all_holiday`

In [26]:
train_stores_holiday = train_stores.merge(all_holiday, on=['date', 'city'], how='left')
train_stores_holiday.head()

del train_stores, all_holiday

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year,city,state,type,cluster,is_holiday
0,66458908,2016-01-01,25,105574,12.0,False,2016,Salinas,Santa Elena,D,1,1.0
1,66458909,2016-01-01,25,105575,9.0,False,2016,Salinas,Santa Elena,D,1,1.0
2,66458910,2016-01-01,25,105857,3.0,False,2016,Salinas,Santa Elena,D,1,1.0
3,66458911,2016-01-01,25,108634,3.0,False,2016,Salinas,Santa Elena,D,1,1.0
4,66458912,2016-01-01,25,108701,2.0,True,2016,Salinas,Santa Elena,D,1,1.0


In [28]:
train_stores_holiday['is_holiday'].unique() # replace nan by 0 ?

array([ 1., nan])

In [94]:
train_stores.shape # (35229871, 11)

(35229871, 11)

In [97]:
train_stores_holiday.shape # (35229871, 12)

(35229871, 12)

> **Same shape rows YES**

In [43]:
train_stores_holiday['is_holiday'] = train_stores_holiday['is_holiday'].fillna(0)

In [44]:
train_stores_holiday['is_holiday'].unique() # replace nan by 0 ?

array([1., 0.])

In [45]:
train_stores_holiday.to_csv('../raw_data/train_stores_holiday.csv')
del train_stores_holiday

---

# Stores data

## Merge 3 data (train, stores, items)

In [ ]:
# train_data
# stores_data
# items_data

In [60]:
train_data.columns

Index(['Unnamed: 0', 'id', 'date', 'store_nbr', 'item_nbr', 'unit_sales',
       'onpromotion', 'year'],
      dtype='object')

In [61]:
items_data.columns

Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object')

In [62]:
stores_data.columns

Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')

In [63]:
train_data.shape # (35229871, 8)

(35229871, 8)

In [7]:
train_items_stores = train_data.merge(items_data, on='item_nbr')\
                                .merge(stores_data, on='store_nbr')
train_items_stores

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year,family,class,perishable,city,state,type,cluster
0,66458908,2016-01-01,25,105574,12.0,False,2016,GROCERY I,1045,0,Salinas,Santa Elena,D,1
1,66505515,2016-01-02,25,105574,16.0,False,2016,GROCERY I,1045,0,Salinas,Santa Elena,D,1
2,66606098,2016-01-03,25,105574,9.0,False,2016,GROCERY I,1045,0,Salinas,Santa Elena,D,1
3,66705153,2016-01-04,25,105574,4.0,False,2016,GROCERY I,1045,0,Salinas,Santa Elena,D,1
4,66800668,2016-01-05,25,105574,6.0,False,2016,GROCERY I,1045,0,Salinas,Santa Elena,D,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35229866,101634524,2016-12-31,28,2036460,2.0,False,2016,DELI,2644,1,Guayaquil,Guayas,E,10
35229867,101309547,2016-12-28,28,2042436,16.0,False,2016,BEVERAGES,1122,0,Guayaquil,Guayas,E,10
35229868,101415706,2016-12-29,28,2042436,94.0,False,2016,BEVERAGES,1122,0,Guayaquil,Guayas,E,10
35229869,101524132,2016-12-30,28,2042436,90.0,False,2016,BEVERAGES,1122,0,Guayaquil,Guayas,E,10


In [67]:
train_items_stores.shape # (35229871, 14)

(35229871, 14)

In [68]:
train_items_stores.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'year', 'family', 'class', 'perishable', 'city', 'state', 'type',
       'cluster'],
      dtype='object')

In [8]:
train_items_stores.to_csv('../raw_data/train_items_stores.csv')
del train_items_stores

---

# Transactions data

---

# Items data

---

# Prepare All merge

## is_special

In [ ]:
# all holiday is_special
# NaN -> False

In [ ]:
holiday_data['is_special'] = True

## Event (column)

In [ ]:
# just assign holiday['type']
# NaN -> Nothing

In [23]:
holiday_data[holiday_data['type'] == 'Event']

,date,type,locale,locale_name,description,transferred
55,2013-05-12,Event,National,Ecuador,Dia de la Madre,False
102,2014-05-10,Event,National,Ecuador,Dia de la Madre-1,False
103,2014-05-11,Event,National,Ecuador,Dia de la Madre,False
106,2014-06-12,Event,National,Ecuador,Inauguracion Mundial de futbol Brasil,False
107,2014-06-15,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Suiza,False
108,2014-06-20,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Honduras,False
113,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False
114,2014-06-28,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False
115,2014-06-29,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False
116,2014-06-30,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False


Check if sales > 0 ? store_is_open : store_is_closed

## is_open

In [ ]:
# with train

In [67]:
# to know if is_open
train_itemsp.loc[train_itemsp['unit_sales'] > 0] # ['is_open'] = True

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year,month,day,day_of_week,family,class,perishable
0,66462415,2016-01-02,2,103665,4.0,False,2016,1,2,5,BREAD/BAKERY,2712,1
1,66464629,2016-01-02,3,103665,16.0,False,2016,1,2,5,BREAD/BAKERY,2712,1
2,66469354,2016-01-02,5,103665,5.0,False,2016,1,2,5,BREAD/BAKERY,2712,1
3,66471373,2016-01-02,6,103665,8.0,False,2016,1,2,5,BREAD/BAKERY,2712,1
4,66473684,2016-01-02,7,103665,4.0,False,2016,1,2,5,BREAD/BAKERY,2712,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9105147,101582835,2016-12-31,2,2042210,3.0,False,2016,12,31,5,PRODUCE,2014,1
9105148,101597360,2016-12-31,8,2042210,26.0,False,2016,12,31,5,PRODUCE,2014,1
9105149,101661404,2016-12-31,42,2042210,14.0,False,2016,12,31,5,PRODUCE,2014,1
9105150,101669025,2016-12-31,45,2042210,13.0,False,2016,12,31,5,PRODUCE,2014,1


make after merge i think

In [ ]:
Holiday : True or False
Event : True or False

Brige : False
Work Day : True

Additional ???
Transfer ???

## Convert Additional

In [ ]:
# Additional to interpret same type everywhere

In [5]:
# holiday_data.loc[holiday_data['type'] == 'Additional'].loc[28]
# for index, one_additional in holiday_data[holiday_data['type'] == 'Additional'].iterrows():
#     print(one_additional['description'][-2])
# holiday_data['type'].loc[28]

holiday_data[holiday_data['type'] == 'Additional']

,date,type,locale,locale_name,description,transferred
28,2012-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
33,2012-12-22,Additional,National,Ecuador,Navidad-3,False
34,2012-12-23,Additional,National,Ecuador,Navidad-2,False
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False
38,2012-12-26,Additional,National,Ecuador,Navidad+1,False
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
53,2013-05-11,Additional,National,Ecuador,Dia de la Madre-1,False
64,2013-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False
81,2013-12-05,Additional,Local,Quito,Fundacion de Quito-1,False


In [6]:
holiday_data[holiday_data['type'] == 'Additional'].shape

(51, 6)

In [7]:
# obtain 'Fundacion de Quito' with slider in Additional in 1 row in description
holiday_data[holiday_data['type'] == 'Additional'].loc[322]['description']

'Fundacion de Guayaquil'

In [8]:
# check day of week
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
holiday_data.date.dt.dayofweek.unique() # 0 = monday ; 6 = sunday

array([4, 6, 3, 5, 0, 1, 2])

**Review loop (Because, except interrupt loop)**

In [11]:
# copy holiday_additional, more safe maybe...
holiday_additional = holiday_data[holiday_data['type'] == 'Additional']

for index, one_additional in holiday_additional.iterrows():
    # check description
    description = one_additional['description']
    # check if convert is possible (Sometimes, description value isn't completed)
    try:
        counter = int(one_additional['description'][-1])
        
        # if '+' or '-'
        if description[-2] == '+':
            # check previous and changed additional type with it
            holiday_data['type'][index] = holiday_data.loc[index - counter]['type']

        elif description[-2] == '-':
            # check next true holiday and same
            holiday_data['type'][index] = holiday_data.loc[index + counter]['type']
        
    except:
        # sometimes, not - and + ... Then assign directly first type founded
        replace_default = description[:-2]
        holiday_data['type'][index] = holiday_data[holiday_data['description'] == description]['type'].iloc[0]

/tmp/ipykernel_1256/3968316913.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holiday_data['type'][index] = holiday_data.loc[index + counter]['type']


In [20]:
# lauch loop twice, i don't know why...
holiday_data.loc[holiday_data['type'] == 'Additional'] # not found normally

,date,type,locale,locale_name,description,transferred


In [19]:
# description with -X or +X, type changed
holiday_data[holiday_data['description'] == 'Navidad-1']

,date,type,locale,locale_name,description,transferred
36,2012-12-24,Holiday,National,Ecuador,Navidad-1,False
88,2013-12-24,Holiday,National,Ecuador,Navidad-1,False
154,2014-12-24,Holiday,National,Ecuador,Navidad-1,False
207,2015-12-24,Holiday,National,Ecuador,Navidad-1,False
293,2016-12-24,Holiday,National,Ecuador,Navidad-1,False
347,2017-12-24,Holiday,National,Ecuador,Navidad-1,False


## Check different type in Holiday (for open)

In [24]:
holiday_data.columns

Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')

make after merge i think

In [ ]:
Holiday : True or False
Event : True or False

Brige : False
Work Day : True

Additional ??? (done)
Transfer ??? (if transferred ? True : False)

# Merge without transactions.csv

In [33]:
train_all_table = train_itemsp.merge(stores_data, on='store_nbr')\
                                .merge(oil_data, on='date')\
                                .merge(all_holiday, on=['date', 'city'], how='left')

del stores_data, oil_data, all_holiday

train_all_table['is_holiday'] = train_all_table['is_holiday'].fillna(0)
train_all_table.head()
train_all_table.to_csv('../raw_data/train_all_table.csv')
del train_all_table

In [36]:
holiday_data.loc[holiday_data['type'] == 'Bridge']

,date,type,locale,locale_name,description,transferred
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False
39,2012-12-31,Bridge,National,Ecuador,Puente Primer dia del ano,False
156,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False
160,2015-01-02,Bridge,National,Ecuador,Puente Primer dia del ano,False
277,2016-11-04,Bridge,National,Ecuador,Puente Dia de Difuntos,False


In [41]:
holiday_data.loc[holiday_data['type'] == 'Bridge'].count()

date           5
type           5
locale         5
locale_name    5
description    5
transferred    5
dtype: int64

In [37]:
holiday_data.loc[holiday_data['type'] == 'Work Day']

,date,type,locale,locale_name,description,transferred
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
149,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
161,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
283,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [40]:
holiday_data.loc[holiday_data['type'] == 'Work Day'].count()

date           5
type           5
locale         5
locale_name    5
description    5
transferred    5
dtype: int64

**check sales in specific date (Work Day)**

In [ ]:
# But : détecter s'il y a eu une vente pendant : Work Day ; Bridge ; Additional ; Event

# Bridge = fermé ; Work Day = ouvert
# Additional : plus de jours en période d'évènement
# Savoir si le magasin était fermé ou pas

In [63]:
holiday_data.loc[holiday_data['type'] == 'Event']

,date,type,locale,locale_name,description,transferred
55,2013-05-12,Event,National,Ecuador,Dia de la Madre,False
103,2014-05-11,Event,National,Ecuador,Dia de la Madre,False
106,2014-06-12,Event,National,Ecuador,Inauguracion Mundial de futbol Brasil,False
107,2014-06-15,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Suiza,False
108,2014-06-20,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Honduras,False
113,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False
114,2014-06-28,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False
115,2014-06-29,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False
116,2014-06-30,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False
117,2014-07-01,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False


In [42]:
holiday_data.loc[holiday_data['type'] == 'Additional']

,date,type,locale,locale_name,description,transferred
28,2012-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
33,2012-12-22,Additional,National,Ecuador,Navidad-3,False
34,2012-12-23,Additional,National,Ecuador,Navidad-2,False
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False
38,2012-12-26,Additional,National,Ecuador,Navidad+1,False
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
53,2013-05-11,Additional,National,Ecuador,Dia de la Madre-1,False
64,2013-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False
81,2013-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
